# Credit Risk Classification

Credit risk poses a classification problem that’s inherently imbalanced. This is because healthy loans easily outnumber risky loans. In this Challenge, you’ll use various techniques to train and evaluate models with imbalanced classes. You’ll use a dataset of historical lending activity from a peer-to-peer lending services company to build a model that can identify the creditworthiness of borrowers.

## Instructions:

This challenge consists of the following subsections:

* Split the Data into Training and Testing Sets

* Create a Logistic Regression Model with the Original Data

* Predict a Logistic Regression Model with Resampled Training Data 

### Split the Data into Training and Testing Sets

Open the starter code notebook and then use it to complete the following steps.

1. Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

2. Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

    > **Note** A value of `0` in the “loan_status” column means that the loan is healthy. A value of `1` means that the loan has a high risk of defaulting.  

3. Check the balance of the labels variable (`y`) by using the `value_counts` function.

4. Split the data into training and testing datasets by using `train_test_split`.

### Create a Logistic Regression Model with the Original Data

Employ your knowledge of logistic regression to complete the following steps:

1. Fit a logistic regression model by using the training data (`X_train` and `y_train`).

2. Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

3. Evaluate the model’s performance by doing the following:

    * Calculate the accuracy score of the model.

    * Generate a confusion matrix.

    * Print the classification report.

4. Answer the following question: How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

### Predict a Logistic Regression Model with Resampled Training Data

Did you notice the small number of high-risk loan labels? Perhaps, a model that uses resampled data will perform better. You’ll thus resample the training data and then reevaluate the model. Specifically, you’ll use `RandomOverSampler`.

To do so, complete the following steps:

1. Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

2. Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

3. Evaluate the model’s performance by doing the following:

    * Calculate the accuracy score of the model.

    * Generate a confusion matrix.

    * Print the classification report.
    
4. Answer the following question: How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

### Write a Credit Risk Analysis Report

For this section, you’ll write a brief report that includes a summary and an analysis of the performance of both machine learning models that you used in this challenge. You should write this report as the `README.md` file included in your GitHub repository.

Structure your report by using the report template that `Starter_Code.zip` includes, and make sure that it contains the following:

1. An overview of the analysis: Explain the purpose of this analysis.


2. The results: Using bulleted lists, describe the balanced accuracy scores and the precision and recall scores of both machine learning models.

3. A summary: Summarize the results from the machine learning models. Compare the two versions of the dataset predictions. Include your recommendation for the model to use, if any, on the original vs. the resampled data. If you don’t recommend either model, justify your reasoning.

In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

import warnings
warnings.filterwarnings('ignore')

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_data_df = pd.read_csv(
    Path("Resources/lending_data.csv"))

# Review the DataFrame
lending_data_df

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600,1
77532,17700.0,10.662,80900,0.629172,11,2,50900,1
77533,17600.0,10.595,80300,0.626401,11,2,50300,1
77534,16300.0,10.068,75300,0.601594,10,2,45300,1


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [12]:
# Separate the data into labels and features

# Separate the y variable, the labels
labels = lending_data_df["loan_status"]

# Separate the X variable, the features
features = lending_data_df[["loan_size","interest_rate","borrower_income","debt_to_income","num_of_accounts","derogatory_marks","total_debt"]]

In [13]:
# Review the y variable Series
labels

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [14]:
# Review the X variable DataFrame
features.head(5)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [16]:
# Check the balance of our target values
labels.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [32]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
training_features, testing_features, training_labels, testing_labels = train_test_split(features, labels)

In [44]:
training_features

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
25659,8900.0,6.919,45700,0.343545,3,0,15700
21445,9900.0,7.353,49800,0.397590,4,0,19800
23148,10200.0,7.459,50800,0.409449,4,1,20800
73559,9300.0,7.097,47400,0.367089,3,0,17400
31389,10200.0,7.442,50600,0.407115,4,1,20600
...,...,...,...,...,...,...,...
10476,8400.0,6.676,43400,0.308756,3,0,13400
49730,8100.0,6.557,42300,0.290780,2,0,12300
10378,9200.0,7.014,46600,0.356223,3,0,16600
31930,9400.0,7.128,47700,0.371069,3,0,17700


In [45]:
testing_features

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
20751,9400.0,7.131,47700,0.371069,3,0,17700
2549,8400.0,6.684,43500,0.310345,3,0,13500
311,9300.0,7.059,47000,0.361702,3,0,17000
29393,9500.0,7.177,48100,0.376299,4,0,18100
59220,12200.0,8.299,58700,0.488927,6,1,28700
...,...,...,...,...,...,...,...
67115,11200.0,7.875,54700,0.451554,5,1,24700
12461,19100.0,11.255,86500,0.653179,12,2,56500
71121,9900.0,7.339,49700,0.396378,4,0,19700
30201,9500.0,7.169,48100,0.376299,4,0,18100


In [46]:
training_labels

25659    0
21445    0
23148    0
73559    0
31389    0
        ..
10476    0
49730    0
10378    0
31930    0
20692    0
Name: loan_status, Length: 58152, dtype: int64

In [47]:
testing_labels

20751    0
2549     0
311      0
29393    0
59220    0
        ..
67115    0
12461    0
71121    0
30201    0
58408    0
Name: loan_status, Length: 19384, dtype: int64

In [ ]:
# Scale the data
scaler = StandardScaler()
training_features_scaled = scaler.fit(training_features)


## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [48]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(training_features, training_labels)
lr_model

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [65]:
# Make a prediction using the testing data
testing_predictions = lr_model.predict(testing_features)

testing_predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [67]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(testing_predictions, testing_labels)

0.9147863891463289

In [74]:
# Generate a confusion matrix for the model
training_matrix = confusion_matrix(testing_labels, testing_predictions)
print(training_matrix)

[[18659   111]
 [   61   553]]


In [78]:
# Print the classification report for the model
class_report = classification_report_imbalanced(testing_labels, testing_predictions)
print(class_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.90      1.00      0.95      0.90     18770
          1       0.83      0.90      0.99      0.87      0.95      0.89       614

avg / total       0.99      0.99      0.90      0.99      0.95      0.90     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model predicted healthy loans 100% of the time, but only predicted high-risk loans 83% of the time.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [79]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
features_resampled, labels_resampled = random_oversampler.fit_resample(training_features, training_labels)

In [80]:
# Count the distinct values of the resampled labels data
labels_resampled.value_counts()

0    56266
1    56266
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [84]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resampled_logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
resampled_logistic_regression_model.fit(features_resampled, labels_resampled)

# Make a prediction using the testing data
resampled_model_prediction = resampled_logistic_regression_model.predict(testing_features)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [85]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(resampled_model_prediction, testing_labels)

0.9184127389769868

In [87]:
# Generate a confusion matrix for the model
training_matrix = confusion_matrix(testing_labels, resampled_model_prediction)
print(training_matrix)

[[18651   119]
 [    3   611]]


In [88]:
# Print the classification report for the model
class_report = classification_report_imbalanced(testing_labels, model_prediction)
print(class_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      1.00      1.00      0.99      0.99     18770
          1       0.84      1.00      0.99      0.91      0.99      0.99       614

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Similarly, the oversampled model predicted healthy loans 100% of the time, but only predicted high-risk loans 84% of the time.